In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
#attempt connection to the database server
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [3]:
#create connection instance
connection = create_server_connection("localhost", "root", "*****")

MySQL Database connection successful


In [4]:
#create a database
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [5]:
#attempt connection to a database on the database server
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [6]:
#run a query on the connection
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [7]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [8]:
def drop_tables():
    drop_sessions = "DROP TABLE Sessions"
    drop_series = "DROP TABLE Series;"
    drop_users = "DROP TABLE Users;"
    drop_devices = "DROP TABLE Devices;"

    execute_query(connection, drop_sessions)
    execute_query(connection, drop_series)
    execute_query(connection, drop_users)
    execute_query(connection, drop_devices)

In [9]:
#create a database for Netflix viewing information
create_database_query = "CREATE DATABASE netflix"
create_database(connection, create_database_query)

Database created successfully


In [10]:
#establish name of database 
db = "netflix"
pw = "*****"

connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [11]:
#create series table with id as key starting with 4 digits, Title, and Movie where 0 represents false and 1 represents true
create_series_table = """
CREATE TABLE Series (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Title VARCHAR(255),
    Movie BIT
);
"""
update_series_increment = """
ALTER TABLE Series AUTO_INCREMENT=1000;
"""

execute_query(connection, create_series_table)
execute_query(connection, update_series_increment)


Query successful
Query successful


In [12]:
#create users table
create_users_table = """
CREATE TABLE Users (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(40)
);
"""
update_users_increment = """
ALTER TABLE Users AUTO_INCREMENT=10;
"""

execute_query(connection, create_users_table)
execute_query(connection, update_users_increment)

Query successful
Query successful


In [13]:
#create devices table
create_devices_table = """
CREATE TABLE Devices (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(255)
);
"""
update_devices_table = """
ALTER TABLE Devices AUTO_INCREMENT=100;
"""

execute_query(connection, create_devices_table)
execute_query(connection, update_devices_table)

Query successful
Query successful


In [14]:
#create sessions table with foreign keys user_id, device_id, and series_id
create_sessions_table = """
CREATE TABLE Sessions (
    id INT PRIMARY KEY AUTO_INCREMENT,
    User_id INT,
    Series_id INT,
    Title VARCHAR(255),
    Start_time TIMESTAMP,
    Duration TIME,
    Device_id INT,
    FOREIGN KEY (User_id) REFERENCES Users(id),
    FOREIGN KEY (Series_id) REFERENCES Series(id),
    FOREIGN KEY (Device_id) REFERENCES Devices(id)
);
"""
update_sessions_table = """
ALTER TABLE Sessions AUTO_INCREMENT=10000;
"""

execute_query(connection, create_sessions_table)
execute_query(connection, update_sessions_table)

Query successful
Query successful


In [15]:
#update unique fields
unique_series = """
ALTER TABLE Series
ADD CONSTRAINT unique_series UNIQUE(Title);
"""
unique_users = """
ALTER TABLE Users
ADD CONSTRAINT unique_users UNIQUE(Name);
"""
unique_devices = """
ALTER TABLE Devices
ADD CONSTRAINT unique_devices UNIQUE(Name);
"""
execute_query(connection, unique_series)
execute_query(connection, unique_users)
execute_query(connection, unique_devices)

Query successful
Query successful
Query successful


In [16]:
def makeLists(results):
    from_db = []

    for result in results:
        result = list(result)
        from_db.append(result)
    
    return from_db

In [17]:
#populate tables with data from Netflix
data = pd.read_csv('Netflix_Data\CONTENT_INTERACTION\ViewingActivity.csv')
data = data[~data['Supplemental Video Type'].str.contains('R', na=False)]
data = data[~data['Supplemental Video Type'].str.contains('H', na=False)]
data['Title'] = data['Title'].str.replace("'", "")
data['Duration'] = pd.to_timedelta(data['Duration'])
data = data[(data['Duration'] > '0 days 00:01:30')]
data['Duration'] = data['Duration'] - pd.to_timedelta(data['Duration'].dt.days, unit='d')
#data.Duration.astype(str).str.replace('0 days ', '')
data['Duration'] = data['Duration'].astype(str).str.split('0 days ').str[-1]

for index, row in data.iterrows():
    title = row['Title']
    series = title
    movie = 1
    user = row['Profile Name']
    device = row['Device Type']
    #determine if the entry is a movie or show
    if "Episode".casefold() in title.casefold():
        movie = 0
        series = title.split(':')[0]
    
    #add info into series, users, and devices tables
    insert_series = "INSERT INTO Series (Title, Movie) VALUES (\'"+series+"\', "+str(movie)+");"
    insert_users = "INSERT INTO Users (Name) VALUES (\'"+user+"\');"
    insert_devices = "INSERT INTO Devices (Name) VALUES (\'"+device+"\');"

    execute_query(connection, insert_series)
    execute_query(connection, insert_users)
    execute_query(connection, insert_devices)

    #get foreign keys for sessions insert
    series_id = makeLists(read_query(connection,"SELECT id FROM Series WHERE Title = \'"+series+"\';"))[0][0]
    users_id = makeLists(read_query(connection,"SELECT id FROM Users WHERE Name = \'"+user+"\';"))[0][0]
    devices_id = makeLists(read_query(connection,"SELECT id FROM Devices WHERE Name = \'"+device+"\';"))[0][0]

    #add session
    insert_session = "INSERT INTO Sessions (User_id, Series_id, Title, Start_time, Duration, Device_id) VALUES ("+str(users_id)+", "+str(series_id)+", \'"+title+"\', \'"+str(row['Start Time'])+"\', \'"+str(row['Duration'])+"\', "+str(devices_id)+")"
    execute_query(connection, insert_session)

Query successful
Query successful
Query successful
Query successful
Error: '1062 (23000): Duplicate entry 'Avatar' for key 'series.unique_series''
Error: '1062 (23000): Duplicate entry 'Alec' for key 'users.unique_users''
Error: '1062 (23000): Duplicate entry 'Roku Fruitland Sound Bar' for key 'devices.unique_devices''
Query successful
Error: '1062 (23000): Duplicate entry 'Avatar' for key 'series.unique_series''
Error: '1062 (23000): Duplicate entry 'Alec' for key 'users.unique_users''
Query successful
Query successful
Query successful
Error: '1062 (23000): Duplicate entry 'Alec' for key 'users.unique_users''
Query successful
Query successful
Error: '1062 (23000): Duplicate entry 'Avatar The Last Airbender' for key 'series.unique_series''
Error: '1062 (23000): Duplicate entry 'Alec' for key 'users.unique_users''
Error: '1062 (23000): Duplicate entry 'Amazon TVAFTBL Smart TV' for key 'devices.unique_devices''
Query successful
Error: '1062 (23000): Duplicate entry 'Avatar' for key 'seri

In [18]:
#turn the tables into readable dataframes
def make_df_series(results):
    from_db = []

    for result in results:
        result = list(result)
        from_db.append(result)

    columns = ['id', 'Title', 'Movie']
    df = pd.DataFrame(from_db, columns=columns)
    df.set_index('id')

    return df

def make_df_users(results):
    from_db = []

    for result in results:
        result = list(result)
        from_db.append(result)

    columns = ['id', 'Name']
    df = pd.DataFrame(from_db, columns=columns)
    df.set_index('id')

    return df

def make_df_devices(results):
    from_db = []

    for result in results:
        result = list(result)
        from_db.append(result)

    columns = ['id', 'Name']
    df = pd.DataFrame(from_db, columns=columns)
    df.set_index('id')

    return df

def make_df_sessions(results):
    from_db = []

    for result in results:
        result = list(result)
        from_db.append(result)

    columns = ['id', 'User_id', 'Series_id', 'Title', 'Start_Time', 'Duration', 'Device_id']
    df = pd.DataFrame(from_db, columns=columns)
    df.set_index('id')

    return df

In [19]:
#display the series table
retrieve = """
SELECT * FROM Series;
"""
display(make_df_series(read_query(connection, retrieve)))

,id,Title,Movie
0,1000,Avatar,0
1,1003,Avatar The Last Airbender,0
2,1029,Glass Onion: A Knives Out Mystery,1
3,1030,ONE PIECE,0
4,1031,Leave the World Behind,1
...,...,...,...
307,11253,Grace and Frankie,0
308,11347,Easy,0
309,11370,Penny Dreadful,0
310,11393,The Host,1


In [20]:
#display the users table
retrieve = """
SELECT * FROM Users;
"""
display(make_df_users(read_query(connection, retrieve)))

,id,Name
0,10,Alec
1,8904,Gwynne


In [21]:
#display the devices table
retrieve = """
SELECT * FROM Devices;
"""
display(make_df_devices(read_query(connection, retrieve)))

,id,Name
0,204,Amazon Fire TV Stick 2020 + Streaming Stick
1,103,Amazon TVAFTBL Smart TV
2,102,Apple iPhone 11 Pro
3,8051,Apple iPhone 6s
4,2967,Apple iPhone 7 (CDMA)
5,2268,Chrome PC (Cadmium)
6,370,Comcast X1 ARM WITH SAGE MVPD STB
7,258,Comcast X1 MIPS WITH SAGE MVPD STB
8,9321,DefaultWidevineAndroidPhone
9,9135,DefaultWidevineAndroidTablets


In [22]:
#display the sessions table
retrieve = """
SELECT * FROM Sessions;
"""
display(make_df_sessions(read_query(connection, retrieve)))

,id,User_id,Series_id,Title,Start_Time,Duration,Device_id
0,10000,10,1000,Avatar: The Last Airbender: Book 1: The Fortun...,2024-03-11 05:30:06,0 days 00:23:43,100
1,10001,10,1000,Avatar: The Last Airbender: Book 1: The Blue S...,2024-03-11 05:19:23,0 days 00:10:39,100
2,10002,10,1000,Avatar: The Last Airbender: Book 1: The Blue S...,2024-03-07 22:23:35,0 days 00:10:46,102
3,10003,10,1003,Avatar The Last Airbender: Season 1: Warriors ...,2024-03-07 20:49:49,0 days 00:21:10,103
4,10004,10,1003,Avatar The Last Airbender: Season 1: Aang (Epi...,2024-03-07 19:51:02,0 days 00:58:43,103
...,...,...,...,...,...,...,...
10390,20390,8904,2677,The Office (U.S.): Season 9: Junior Salesman (...,2017-07-13 23:53:53,0 days 00:03:45,2967
10391,20391,8904,2677,The Office (U.S.): Season 9: Vandalism (Episod...,2017-07-13 06:04:08,0 days 00:08:27,2967
10392,20392,8904,2677,The Office (U.S.): Season 9: Junior Salesman (...,2017-07-13 05:42:30,0 days 00:21:31,2967
10393,20393,8904,11393,The Host,2017-06-24 02:04:07,0 days 02:05:14,370


In [23]:
#run to clear the database
'''
sql = """
DROP DATABASE netflix;
"""
execute_query(connection,sql)
'''
#drop_tables()

'\nsql = """\nDROP DATABASE netflix;\n"""\nexecute_query(connection,sql)\n'